# CIFAR-10 Classification
> Performance analysis for CIFAR-10 Classification on all hardware platforms

- toc: true 
- badges: true
- comments: true
- categories: [CIFAR-10,Rooflines,Performance Prediction]
- image: images/cifar_logo.png

In [1]:
#hide
#from altair.vegalite.v4.api import Selection
import pandas as pd
import numpy as np
import altair as alt

W = 600
H = 480

pd.options.display.max_rows = 500 # this will set limit of rows to 500
pd.options.display.max_columns = 500 # this will set limit of columns to 500

pd.options.display.float_format = '{:20,.4f}'.format

csv_path = "./data/cleaned_csv/backup.csv"

%run scripts/altair_plots.py
%run scripts/overlapped_pareto.py
%run scripts/script_tables.py 

# Theretical Analysis of CIFAR-10

### Rooflines for All Hardware Platforms and CNNs

Combining application requirements with hardware platform characteristics can be leveraged for performance predictions using UCB’s roofline models. Using assumptions for where weights, activation tensors, and state of a neural network are stored, combined with the size of the datatypes used, allow us to derive the arithmetic intensity of a neural network during inference. Combined with the roofline for a given hardware platform, we can provide insight as to whether a neural network will be memory or compute bound and guidance for what is theoretically possible in regards to its throughput.

In [2]:
#hide_input
# Run the Rooflines script which processes the data and creates the chart
%run scripts/altair_plots.py
rooflines('cifar')

alt.LayerChart(...)

### Performance Prediction

The following heatmap shows the theoretical performance for the listed hardware platforms for CIFAR-10 classification. The metric used for the theoretical performance is input/second.
We observe that prunning along with quantization outputs some of the best performance results.

In [3]:
#hide_input
%run scripts/altair_plots.py
heatmap('data/performance_predictions_imagenet_mnist_cifar.csv', 'cifar', 'Performance Prediction for CIFAR-10')

alt.LayerChart(...)

# Experimental Data Analysis

### Overview of All Measurements for CIFAR-10

In this table, within the rows, we show the type of hardware platforms that we used for this task (for example FPGA or GPU) and then more specifically the exact name of the different hardware platforms. For each hardware platform, we list the sweep of specific deployment parameters (batch sizes, operating modes etc) that were used for the experimentation in separate columns. In the columns, we show CNN topologies. When a CNN topology was implemented on a given hardware platform, we show in the corresponding cell the precisions (quantization information) and the channel pruning scale. Otherwise, “na” indicates that the topology wasn’t executed on this specific hardware platform. Many combinations between topology and hardware platform are not supported by the vendors dedicated software environments. INTx depicts a fixed point integer representation with x bits. FPy represents a floating point representation with y bits, for example FP32 is singe precision floating point. Table follows below.

In [4]:
#hide
%run scripts/altair_plots.py   #get table with the experiments overview
print(pd.read_csv('data/overview_experiments_cifar10.csv').to_markdown())

|    | Hardware   | Platform         | CNV                              | Batch/Stream/Thread                  |
|---:|:-----------|:-----------------|:---------------------------------|:-------------------------------------|
|  0 | FPGA       | ZCU102-DPU       | na                               | [1,2,3,4,5,6,7,8]                    |
|  1 | FPGA       | ZCU104-DPU       | na                               | [1,2,3,4,5,6,7,8]                    |
|  2 | FPGA       | Ultra96-DPU      | na                               | [1,2,3,4,5,6,7,8]                    |
|  3 | FPGA       | ZCU104-FINN      | [INT2,INT4]*[100%,50%,25%,12.5%] | [1,2,4,8,16,32,64,128,256,512,10000] |
|  4 | FPGA       | ZCU104-BISMO     | [INT2,INT4]*[100%,50%,25%,12.5%] | [2,4,8,16,32,64,128]                 |
|  5 | GPU        | TX2-maxn         | [FP16,FP32]*[100%,50%,25%,12.5%] | [1,2,4,8,16,32,64,128]               |
|  6 | GPU        | TX2-maxp         | [FP16,FP32]*[100%,50%,25%,12.5%] | [1,2,4,8,16,32,64,128]

In [5]:
#hide_input
%run scripts/script_tables.py 
#get table with the experiments overview
dataframes = csv_to_dataframe_multiindex(['data/overview_experiments_cifar10_.csv'])
for dataframe in dataframes:   
       display(HTML(dataframe.to_html(index=False)))

In [6]:
#hide
master_df = pd.read_csv(csv_path)
cnv_df = master_df.copy()
#is_maxp = lambda row: row.HWType != "GPU" or row["Op mode"].split(",")[0] == "maxp"
#maxp_df = master_df[master_df.apply(is_maxp, axis=1)]
cnv_df['HWType'] = cnv_df['HWType'] + ('-' + cnv_df['Op mode']).fillna('')
cnv_df["hw_datatype_prun_net"] = cnv_df.apply(lambda r: "_".join([r.HWType, r.Datatype, r.PruningFactor, r.NN_Topology]), axis=1)
cnv_df = cnv_df[(cnv_df["NN_Topology"] == "CNV") & cnv_df['lat-comp'].notna() & cnv_df["top1 [%]"].notna()]
cnv_df["hw_datatype_prun_net"] = cnv_df.apply(lambda r: "_".join([r.HWType, r.Datatype, r.PruningFactor, r.NN_Topology]), axis=1)
cnv_df["PruningFactor"] = cnv_df["PruningFactor"].str.strip("%").astype(float)
norm_by_group(cnv_df, "lat-comp", "NN_Topology");
cnv_df["datatype_model"] = cnv_df.Datatype + '_' + cnv_df.HWType
cnv_df["tag"] = cnv_df.apply(lambda r: "_".join([r.HWType, r.Datatype, r.NN_Topology, str(r.PruningFactor)]), axis=1)
cnv_df.rename(columns={"top1 [%]": "top1"}, inplace=True)

#fill GOPS gap
cnv_df['GOPS'] = cnv_df.apply(lambda r: 0.469450 if r.PruningFactor == 100.00 else 
                                          (0.118923636 if r.PruningFactor == 50.00 else 
                                           (0.030511732 if r.PruningFactor == 25.00 else 0.008018676 )) , axis=1)

#fill in tp-system and tp-cmp
cnv_df['tp-system'] = cnv_df['fps-system'] * cnv_df['GOPS']
cnv_df['tp-comp'] = cnv_df['fps-comp'] * cnv_df['GOPS']
cnv_df['GOPS'] = cnv_df['GOPS'] * cnv_df['batch/thread/stream']

#save it all up
cnv_df.to_csv('data/cleaned_csv/experimental_data_cifar.csv', index = False)

cnv_df

,NN_Topology,HWType,Datatype,Op mode,batch/thread/stream,lat-sys,lat-comp,fps-system,fps-comp,tp-system,tp-comp,top1,top5 [%],Base_Pwr_W,Idle_Pwr_W,Full_Pwr_W,GOPS,PruningFactor,level,hw_peak_perf,hw_bandwidth,nn_total_operations,hw_datatype_prun_net,norm-lat-comp,datatype_model,tag
8,CNV,NCS,FP16,NaN,1,3.4,1.9000,293.8120,526.3050,2.3560,4.2203,77.8200,nan,0.5300,1.2000,1.5600,0.0080,12.5000,l3,0.5000,12.8000,0.0100,NCS_FP16_12.50%_CNV,0.0001,FP16_NCS,NCS_FP16_CNV_12.5
9,CNV,NCS,FP16,NaN,2,5.61,2.5536,356.5690,783.1970,2.8592,6.2802,77.8200,nan,0.5300,1.2000,1.5800,0.0160,12.5000,l3,0.5000,12.8000,0.0100,NCS_FP16_12.50%_CNV,0.0002,FP16_NCS,NCS_FP16_CNV_12.5
10,CNV,NCS,FP16,NaN,4,10.09,3.9493,396.4320,"1,012.8400",3.1789,8.1216,77.8200,nan,0.5300,1.2000,1.5900,0.0321,12.5000,l3,0.5000,12.8000,0.0100,NCS_FP16_12.50%_CNV,0.0003,FP16_NCS,NCS_FP16_CNV_12.5
11,CNV,NCS,FP16,NaN,8,19.18,6.9201,417.1720,"1,156.0600",3.3452,9.2701,77.8200,nan,0.5300,1.2000,1.5900,0.0641,12.5000,l3,0.5000,12.8000,0.0100,NCS_FP16_12.50%_CNV,0.0005,FP16_NCS,NCS_FP16_CNV_12.5
12,CNV,NCS,FP16,NaN,16,37.01,12.6046,432.2930,"1,269.3800",3.4664,10.1787,77.8200,nan,0.5300,1.2000,1.5800,0.1283,12.5000,l3,0.5000,12.8000,0.0100,NCS_FP16_12.50%_CNV,0.0009,FP16_NCS,NCS_FP16_CNV_12.5
13,CNV,NCS,FP16,NaN,32,72.74,23.6896,439.9120,"1,350.8000",3.5275,10.8316,77.8200,nan,0.5300,1.2000,1.5800,0.2566,12.5000,l3,0.5000,12.8000,0.0100,NCS_FP16_12.50%_CNV,0.0016,FP16_NCS,NCS_FP16_CNV_12.5
14,CNV,NCS,FP16,NaN,64,143.09,45.6772,447.2780,"1,401.1400",3.5866,11.2353,77.8200,nan,0.5300,1.2000,1.5900,0.5132,12.5000,l3,0.5000,12.8000,0.0100,NCS_FP16_12.50%_CNV,0.0031,FP16_NCS,NCS_FP16_CNV_12.5
15,CNV,NCS,FP16,NaN,128,284.73,89.2251,449.5420,"1,434.5700",3.6047,11.5034,77.8200,nan,0.5300,1.2000,1.5800,1.0264,12.5000,l3,0.5000,12.8000,0.0100,NCS_FP16_12.50%_CNV,0.0061,FP16_NCS,NCS_FP16_CNV_12.5
16,CNV,NCS,FP16,NaN,1,3.5,1.9543,285.5170,511.6840,8.7116,15.6124,83.2800,nan,0.5300,1.2000,1.5900,0.0305,25.0000,l3,0.5000,12.8000,0.0300,NCS_FP16_25%_CNV,0.0001,FP16_NCS,NCS_FP16_CNV_25.0
17,CNV,NCS,FP16,NaN,2,5.8,2.8063,344.8740,712.6720,10.5227,21.7449,83.2800,nan,0.5300,1.2000,1.6200,0.0610,25.0000,l3,0.5000,12.8000,0.0300,NCS_FP16_25%_CNV,0.0002,FP16_NCS,NCS_FP16_CNV_25.0


In [7]:
#hide
figa_df = cnv_df[(cnv_df["HWType"].isin(["NCS", "ZCU104-Bismo", "U96-Quadcore A53"]))]
figb_df = cnv_df[(cnv_df["HWType"].isin(["GPU", "ZCU104-FINN", "U96-Quadcore A53"]))]

### Line Plot

In [8]:
#hide_input
fig25s = []
fig25_dfs = [figa_df, figb_df]
for df in fig25_dfs:
    sel = alt.selection_multi(fields=["hw_datatype_prun_net"], bind="legend")
    fig25_dot = alt.Chart(df).mark_point().encode(
        x='lat-comp',
        y=alt.Y('fps-comp', scale=alt.Scale(type="log")),
        color=select_color(sel, 'hw_datatype_prun_net:N'),
        tooltip=['fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
    )
    fig25_line = alt.Chart(df).mark_line().encode(
        x='lat-comp',
        y='fps-comp',
        color=select_color(sel, 'hw_datatype_prun_net:N'),
        tooltip=['fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
    )

    fig = (fig25_dot+fig25_line).properties(
        title="Latency Versus Performance for Pruned and Quantized CNV Variants",
        width=W/len(fig25_dfs),
        height=H,
    ).add_selection(sel).interactive()
    
    fig25s.append(fig)
    
alt.hconcat(*fig25s)

alt.HConcatChart(...)

### Boxplots

In [9]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=cnv_df, 
        xaxis='PruningFactor', 
        yaxis="lat-comp", 
        color_col= 'PruningFactor', 
        facet_column='datatype_model', 
        title="Latency by Hardware/Framework and Pruning for CIFAR-10 Classification")

alt.FacetChart(...)

In [10]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=cnv_df, 
        xaxis='PruningFactor', 
        yaxis="fps-comp", 
        color_col= 'PruningFactor', 
        facet_column='datatype_model', 
        title="Throughput by Hardware/Framework and Pruning for  CIFAR-10 Classification")

alt.FacetChart(...)

In [11]:
#hide
# First delete datapoints that are only one value, so the plot doesnt appear empty
cnv_df_tmp = cnv_df
cnv_df_tmp = delete_unique_values(df= cnv_df_tmp, 
                                 col_a= 'HWType', 
                                 col_b='Full_Pwr_W' )

C:\Users\alinav\Documents\GitHub\QutibenchWeb\_notebooks\scripts\altair_plots.py:653: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['count'] = df.groupby(col_a)[col_a].transform('count')


In [12]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=cnv_df_tmp, 
        xaxis='PruningFactor', 
        yaxis="Full_Pwr_W", 
        color_col= 'PruningFactor', 
        facet_column='datatype_model', 
        title="Power Consumption by Hardware/Framework and Pruning CIFAR-10 Classification")

alt.FacetChart(...)

### Pareto Graphs

The following pareto graph presents the accuracy versus performance in fps for all the Hardware Platforms across different Pruning and Quantization configurations. This provides insights into accuracy-based comparisons.

In [13]:
#hide_input
%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
pareto_graph(df= cnv_df, 
             groupcol= 'hw_datatype_prun_net',
             xcol = 'fps-comp', 
             ycol= 'top1',
             W= W, 
             H= H, 
             title= "CNV Classification Design Space: Accuracy Versus Performance")

alt.LayerChart(...)

# Theoretical Pareto and Measured Pareto Overlapped

In order to easily understand how accurate predictions were, an overlapping between the Theoretical Pareto Plot and Measured Pareto Plot was made. In the plot below we have both theoretical (orange) and measured (blue) pareto lines. All measured datapoins are represented as crosses and all theoretical datatpoins are represented as circles. Some theoretical datapoints don't have a measured matched datapoint and the same goes for the measured datapoints. The theoretical pareto curve is, as expected, on the right of the measured one, as predictions are sometimes different form measurements.

In [14]:
#hide_input
%run scripts/overlapped_pareto.py
get_overlapped_pareto('cifar','Overlapped Pareto Plots Theoretical + Measured for CIFAR-10') 

alt.LayerChart(...)

# Efficiency Plot

In order to understand the gap between the theoretical predictions and what was measured, an efficiency bar-chart was created. The size of the bar reflects the absolute performance, whereby all theoretical predictions are shown in red, theoretical peak performance in blue, and all measured datapoints in orange. The orange bars are annotated with the efficiency achieved as a percentage of the predicted performance. Please note the logarithmic y-axis scale. The theoretical predictions take memory bottlenecks into account, as such measured performance can actually exceed the predicted result, in which case the percentage can be above 100%.

In [15]:
#hide_input
%run scripts/overlapped_pareto.py
efficiency_plot(machine_learning_task= 'cifar', title='Efficiency Plots for CIFAR-10')

alt.FacetChart(...)

alt.FacetChart(...)

alt.FacetChart(...)

In [16]:
#hide
cnv_df.head()

,NN_Topology,HWType,Datatype,Op mode,batch/thread/stream,lat-sys,lat-comp,fps-system,fps-comp,tp-system,tp-comp,top1,top5 [%],Base_Pwr_W,Idle_Pwr_W,Full_Pwr_W,GOPS,PruningFactor,level,hw_peak_perf,hw_bandwidth,nn_total_operations,hw_datatype_prun_net,norm-lat-comp,datatype_model,tag
8,CNV,NCS,FP16,NaN,1,3.4,1.9000,293.8120,526.3050,2.3560,4.2203,77.8200,nan,0.5300,1.2000,1.5600,0.0080,12.5000,l3,0.5000,12.8000,0.0100,NCS_FP16_12.50%_CNV,0.0001,FP16_NCS,NCS_FP16_CNV_12.5
9,CNV,NCS,FP16,NaN,2,5.61,2.5536,356.5690,783.1970,2.8592,6.2802,77.8200,nan,0.5300,1.2000,1.5800,0.0160,12.5000,l3,0.5000,12.8000,0.0100,NCS_FP16_12.50%_CNV,0.0002,FP16_NCS,NCS_FP16_CNV_12.5
10,CNV,NCS,FP16,NaN,4,10.09,3.9493,396.4320,"1,012.8400",3.1789,8.1216,77.8200,nan,0.5300,1.2000,1.5900,0.0321,12.5000,l3,0.5000,12.8000,0.0100,NCS_FP16_12.50%_CNV,0.0003,FP16_NCS,NCS_FP16_CNV_12.5
11,CNV,NCS,FP16,NaN,8,19.18,6.9201,417.1720,"1,156.0600",3.3452,9.2701,77.8200,nan,0.5300,1.2000,1.5900,0.0641,12.5000,l3,0.5000,12.8000,0.0100,NCS_FP16_12.50%_CNV,0.0005,FP16_NCS,NCS_FP16_CNV_12.5
12,CNV,NCS,FP16,NaN,16,37.01,12.6046,432.2930,"1,269.3800",3.4664,10.1787,77.8200,nan,0.5300,1.2000,1.5800,0.1283,12.5000,l3,0.5000,12.8000,0.0100,NCS_FP16_12.50%_CNV,0.0009,FP16_NCS,NCS_FP16_CNV_12.5


# CIFAR-10 Power Measurements

In [23]:
#hide_input
def faceted_bar_chart(df,xcol,ycol,colorcol,textcol,title,columncol):
    bars = alt.Chart().mark_bar().encode(
        x=alt.X(xcol +':N', title=''),
        y=alt.Y(ycol + ':Q',  title='Power [W]'),
        color=alt.Color(colorcol +':N', title='Hardware'),
    )
    text = bars.mark_text(
        angle=270,
        align='left',
        baseline='middle',
        dx=10  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text= alt.Text(ycol + ':Q', format='.1f')
    )
    return alt.layer(bars, text, data=df).facet(
        column=alt.Column(columncol+':N', header=alt.Header(labelAngle=-85, labelAlign='right'), title=title)
    ).interactive()

df = cnv_df.copy()
df = df.loc[:,['NN_Topology','HWType','Datatype','Full_Pwr_W','PruningFactor']]
df['NN_Topology'] = df.NN_Topology + '_' + df.PruningFactor.astype(str)
df['hardw_opmode_datatype'] =df.HWType  +'_'+ df.Datatype
df = df.groupby(['NN_Topology','HWType','Datatype','hardw_opmode_datatype'])['Full_Pwr_W'].mean().reset_index()

faceted_bar_chart(df=df, xcol='NN_Topology', ycol='Full_Pwr_W', colorcol='hardw_opmode_datatype', textcol='Full_Pwr_W',columncol = 'hardw_opmode_datatype', title='Power Plots for CIFAR-10 Classification')



alt.FacetChart(...)

In [22]:
#hide_input
def faceted_bar_chart(df,xcol,ycol,colorcol,textcol,title,columncol):
    bars = alt.Chart().mark_bar().encode(
        x=alt.X(xcol +':N', title=''),
        y=alt.Y( ycol + ':Q',  title='Power [mW]'),
        color=alt.Color(colorcol +':N'),
    )
    
    text = bars.mark_text(
        angle=270,
        align='left',
        baseline='middle',
        dx=10  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text= alt.Text(ycol + ':Q', format='.1f')
    )
    return alt.layer(bars, text, data=df).interactive().facet(
        column=alt.Column(columncol+':N',  header=alt.Header(labelAngle=-85, labelAlign='right'), title=title))

df = cnv_df.copy()
df = df.loc[:,['NN_Topology','HWType','Datatype','Full_Pwr_W','PruningFactor','fps-comp']]
df['NN_Topology'] = df.NN_Topology + '_' + df.PruningFactor.astype(str)
df['fps/power'] = df.Full_Pwr_W / df['fps-comp'] *1000
df['hardw_opmode_datatype'] =df.HWType  +'_'+ df.Datatype
df = df.groupby(['NN_Topology','HWType','Datatype','hardw_opmode_datatype'])['fps/power'].mean().reset_index()

faceted_bar_chart(df=df, xcol='NN_Topology', ycol='fps/power', colorcol='hardw_opmode_datatype', textcol='Full_Pwr_W',columncol = 'hardw_opmode_datatype', title='FPS nomalized by Power Plots for CIFAR-10 Classification')



alt.FacetChart(...)